# MIS: Clase complementaria 12 (2023-20)

En esta clase se estudiará el algoritmo de Louvain y tendrá un trabajo grupal.

#### Parte 1: Entienda lo que está escrito.

In [26]:
import networkx as nx
import numpy as np
import copy


In [27]:
def getting_ready(graph):
    # Crea el listado de nodos
    nodes = graph.nodes
    # Crea matriz de adyacencia
    A = nx.adjacency_matrix(graph).todense()
    # Crea vector de k's
    Ks = A.sum(1)
    # Crea m
    m = ((Ks.sum(0))/2)[0,0]
    # Crea las comunidades e identificadores de nodos con su comunidad
    Communities = {"C_{}".format(i):[i] for i in nodes}
    Identifier = {i:"C_{}".format(i) for i in nodes}
    C = {"Com":Communities,"Id":Identifier}
    return A,Ks,m,C


In [28]:
def new_partition(C,node,community):
    CC = copy.deepcopy(C)
    Com_In = "C_{}".format(community)
    Com_Out = CC["Id"][node]
    # Se modifican las comunidades
    CC["Com"][Com_In].append(node)
    CC["Com"][Com_Out].remove(node)
    # Se modifica el id de la comunidad en la que node está
    CC["Id"][node] = Com_In
    # Se entrega la nueva partición
    return CC 

In [29]:
def sigmas(A,C,community):
    # Se inicializan los sigmas
    sigma_in = 0
    sigma_tot = 0
    # Se obtiene el identificador de la comunidad
    Com = "C_{}".format(community)
    # Se obtienen los miembros de la comunidad en cuestión
    members = C["Com"][Com]
    # Se recorren los miembros para actualizar los sigmas
    for member in members:
        sigma_in += A[members,member].sum(0)[0,0]
        sigma_tot += A[:,member].sum(0)[0,0]
    return sigma_in,sigma_tot


In [30]:
def f_k_i_in(A,C,node,community):
    # Se obtiene el identificador de la comunidad
    Com = "C_{}".format(community)
    # Se obtienen los miembros de la comunidad en cuestión
    members = C["Com"][Com]
    # Se obtiene lo que necesitamos
    k_i_in = A[node,members].sum(1)[0,0]
    return k_i_in

In [31]:
def modularity_gain(A,C,node,community,Ks,m):
    # Se obtienen los componentes de la fórmula
    sigma_in,sigma_tot = sigmas(A,C,community)
    k_i_in = f_k_i_in(A,C,node,community)
    k_i = Ks[node,0]
    # Se construye la fórmula
    den =2*m
    term_1 = (sigma_in+k_i_in)/den - ((sigma_tot+k_i)/den)**2 
    term_2 = (sigma_in/den)-(sigma_tot/den)**2-(k_i/den)**2
    # Delta
    DQ = term_1-term_2
    return DQ


In [32]:
def assign_to_community(graph,A,C,node,Ks,m):
    # Se obtienen los vecinos del nodo node:
    vecinos = nx.neighbors(graph,node)
    # Se encuentran los gains:
    gains = {j:modularity_gain(A,C,node,j,Ks,m) for j in vecinos}
    # Se optimiza y se saca la comunidad a la que queremos enviar al nodo node:
    community = max(gains, key=gains.get)

    # Se actualizan las comunidades
    Com_In = "C_{}".format(community)
    Com_Out = C["Id"][node]
    # Se modifican las comunidades
    C["Com"][Com_In].append(node)
    C["Com"][Com_Out].remove(node)
    # Se modifica el id de la comunidad en la que node está
    C["Id"][node] = Com_In

    # Se retornan los valores relevantes
    return C

#### Parte 2: Use las funciones anteriores para crear el algoritmo.

Recuerde, en este punto ustedes deben entender y plantear el algoritmo según les parezca.